In [6]:
import ast

In [7]:
source_code = """def add(a, b):
    return a + b
"""

In [8]:
tree = ast.parse(source_code)

In [9]:
ast.dump(tree)


"Module(body=[FunctionDef(name='add', args=arguments(posonlyargs=[], args=[arg(arg='a'), arg(arg='b')], kwonlyargs=[], kw_defaults=[], defaults=[]), body=[Return(value=BinOp(left=Name(id='a', ctx=Load()), op=Add(), right=Name(id='b', ctx=Load())))], decorator_list=[])], type_ignores=[])"

In [10]:
print(ast.dump(tree, indent=2))

Module(
  body=[
    FunctionDef(
      name='add',
      args=arguments(
        posonlyargs=[],
        args=[
          arg(arg='a'),
          arg(arg='b')],
        kwonlyargs=[],
        kw_defaults=[],
        defaults=[]),
      body=[
        Return(
          value=BinOp(
            left=Name(id='a', ctx=Load()),
            op=Add(),
            right=Name(id='b', ctx=Load())))],
      decorator_list=[])],
  type_ignores=[])


In [ ]:
 Module(
  body=[
    FunctionDef(
      name='add',
      args=arguments(
        posonlyargs=[],
        args=[
          arg(arg='a'),
          arg(arg='b')],
        kwonlyargs=[],
        kw_defaults=[],
        defaults=[]),
      body=[
        Return(
          value=BinOp(
            left=Name(id='a', ctx=Load()),
            op=Add(),
            right=Name(id='b', ctx=Load())))],
      decorator_list=[])],
  type_ignores=[])

In [28]:
for node in ast.walk(tree):
    print(f"""
Node Type: {type(node).__name__}
Node Body: {getattr(node, 'body', None)}
          """)
    for field, value in ast.iter_fields(node):
        print(f"    {field}: {value}")



Node Type: Module
Node Body: [<ast.FunctionDef object at 0x1058c5e40>]
          
    body: [<ast.FunctionDef object at 0x1058c5e40>]
    type_ignores: []

Node Type: FunctionDef
Node Body: [<ast.Return object at 0x1058c5a50>]
          
    name: add
    args: <ast.arguments object at 0x1058c5db0>
    body: [<ast.Return object at 0x1058c5a50>]
    decorator_list: []
    returns: None
    type_comment: None

Node Type: arguments
Node Body: None
          
    posonlyargs: []
    args: [<ast.arg object at 0x1058c5a80>, <ast.arg object at 0x1058c5a20>]
    vararg: None
    kwonlyargs: []
    kw_defaults: []
    kwarg: None
    defaults: []

Node Type: Return
Node Body: None
          
    value: <ast.BinOp object at 0x1058c59f0>

Node Type: arg
Node Body: None
          
    arg: a
    annotation: None
    type_comment: None

Node Type: arg
Node Body: None
          
    arg: b
    annotation: None
    type_comment: None

Node Type: BinOp
Node Body: None
          
    left: <ast.Name o

In [20]:
dir(node.body[0])

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__match_args__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_attributes',
 '_fields',
 'args',
 'body',
 'col_offset',
 'decorator_list',
 'end_col_offset',
 'end_lineno',
 'lineno',
 'name',
 'returns',
 'type_comment']

# AST-Based Search Index (Cline-Inspired)

This implementation extracts definitions from Python files and builds a searchable index.
Inspired by Cline's approach:
- Extract only function/class signatures (first line)
- Build symbol → file mapping
- Fast search without reading full implementations

In [29]:
import ast
import os
from pathlib import Path
from typing import Dict, List, Tuple
from dataclasses import dataclass

@dataclass
class Definition:
    """Represents a code definition (function, class, method)"""
    name: str
    type: str  # 'function', 'class', 'method'
    signature: str  # First line only (Cline-style)
    file_path: str
    line_number: int
    parent: str = None  # For methods, the class name

class ASTSearchIndex:
    """AST-based search index for Python code"""
    
    def __init__(self):
        self.definitions: List[Definition] = []
        self.symbol_map: Dict[str, List[Definition]] = {}  # symbol name -> definitions, contains all the definitions for this one symbol
    
    def extract_definitions(self, file_path: str) -> List[Definition]:
        """Extract function/class definitions from a Python file (Cline-inspired)"""
        definitions = []
        
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                source = f.read()
            
            tree = ast.parse(source)
            lines = source.split('\n')
            
            for node in ast.walk(tree):
                if isinstance(node, ast.FunctionDef):
                    # Get first line of function (signature only)
                    sig_line = lines[node.lineno - 1].strip()
                    
                    # Check if it's a method (has parent class)
                    parent_class = self._find_parent_class(tree, node)
                    
                    definitions.append(Definition(
                        name=node.name,
                        type='method' if parent_class else 'function',
                        signature=sig_line,
                        file_path=file_path,
                        line_number=node.lineno,
                        parent=parent_class
                    ))
                
                elif isinstance(node, ast.ClassDef):
                    # Get first line of class definition
                    sig_line = lines[node.lineno - 1].strip()
                    
                    definitions.append(Definition(
                        name=node.name,
                        type='class',
                        signature=sig_line,
                        file_path=file_path,
                        line_number=node.lineno
                    ))
        
        except Exception as e:
            print(f"Error parsing {file_path}: {e}")
        
        return definitions
    
    def _find_parent_class(self, tree: ast.Module, func_node: ast.FunctionDef) -> str:
        """Find the parent class of a function node"""
        for node in ast.walk(tree):
            if isinstance(node, ast.ClassDef):
                if func_node in node.body:
                    return node.name
        return None
    
    def index_file(self, file_path: str):
        """Index a single Python file"""
        defs = self.extract_definitions(file_path)
        
        for definition in defs:
            self.definitions.append(definition)
            
            # Build symbol map
            if definition.name not in self.symbol_map:
                self.symbol_map[definition.name] = []
            self.symbol_map[definition.name].append(definition)
    
    def index_directory(self, directory: str, max_files: int = 50):
        """Index all Python files in a directory (with Cline's 50-file limit)"""
        py_files = list(Path(directory).rglob('*.py'))[:max_files]
        
        print(f"Indexing {len(py_files)} Python files...")
        
        for file_path in py_files:
            self.index_file(str(file_path))
        
        print(f"Indexed {len(self.definitions)} definitions")
    
    def search(self, query: str) -> List[Definition]:
        """Search for symbols matching query"""
        results = []
        
        # Exact match
        if query in self.symbol_map:
            results.extend(self.symbol_map[query])
        
        # Partial match (case-insensitive)
        query_lower = query.lower()
        for name, defs in self.symbol_map.items():
            # If the query is a substring of the name, and not an exact match
            if query_lower in name.lower() and query != name:
                results.extend(defs)
        
        return results
    
    def format_results(self, results: List[Definition]) -> str:
        """Format search results in Cline's pipe-delimited style"""
        if not results:
            return "No definitions found."
        
        output = []
        current_file = None
        
        # Group by file
        results_by_file = {}
        for result in results:
            if result.file_path not in results_by_file:
                results_by_file[result.file_path] = []
            results_by_file[result.file_path].append(result)
        
        # Format output
        for file_path, defs in results_by_file.items():
            output.append(f"\n|---- {file_path}")
            
            for definition in sorted(defs, key=lambda d: d.line_number):
                indent = "  " if definition.parent else ""
                output.append(f"{indent}{definition.signature}")
        
        return '\n'.join(output)
    
    def print_summary(self):
        """Print index summary statistics"""
        classes = sum(1 for d in self.definitions if d.type == 'class')
        functions = sum(1 for d in self.definitions if d.type == 'function')
        methods = sum(1 for d in self.definitions if d.type == 'method')
        
        print(f"\n📊 Index Summary:")
        print(f"  Classes: {classes}")
        print(f"  Functions: {functions}")
        print(f"  Methods: {methods}")
        print(f"  Total: {len(self.definitions)}")
        print(f"  Unique symbols: {len(self.symbol_map)}")

## Example 1: Index the current project

In [32]:
%cd /Users/shishirjoshi/development/lab/coding-agent

/Users/shishirjoshi/development/lab/coding-agent


/opt/anaconda3/envs/ask-clari/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [33]:
# Create index and scan the agent directory
index = ASTSearchIndex()
index.index_directory('agent')
index.print_summary()

Indexing 11 Python files...
Error parsing agent/ui.py: f-string expression part cannot include a backslash (<unknown>, line 39)
Indexed 72 definitions

📊 Index Summary:
  Classes: 11
  Functions: 13
  Methods: 48
  Total: 72
  Unique symbols: 69


## Example 2: Search for symbols

In [34]:
# Search for "chat" - should find Agent.chat and related functions
results = index.search('chat')
print(index.format_results(results))


|---- agent/agent_loop.py
  def chat(self, user_text: str) -> str:

|---- agent/llm_openai_compat.py
  def chat(self, messages: list[dict[str, Any]], tools: list[dict[str, Any]]) -> dict[str, Any]:
  def _responses_to_chat_message(resp: dict[str, Any]) -> dict[str, Any]:


In [35]:
# Search for "Agent" - should find the Agent class
results = index.search('Agent')
print(index.format_results(results))


|---- agent/agent_loop.py
class AgentConfig:
class Agent:


In [36]:
# Search for "debug" - partial match
results = index.search('debug')
print(index.format_results(results))


|---- agent/agent_loop.py
  def _debug_render_md(self, text: str) -> str:
  def _get_debug_theme(self):
  def _debug_prefix(self, role: str | None = None) -> str:
  def _debug_role(self, role: str | None) -> str:
  def _debug_label(self, label: str, *, kind: str = "dim") -> str:
  def _debug_print_round_header(self, round_idx: int) -> None:
  def _debug_print_request_summary(self) -> None:
  def _debug_print_response_summary(self, assistant_msg: dict[str, Any]) -> None:


In [40]:
source_code = """def add(a, b): return a + b"""

print(ast.dump(ast.parse(source_code), indent=2))

Module(
  body=[
    FunctionDef(
      name='add',
      args=arguments(
        posonlyargs=[],
        args=[
          arg(arg='a'),
          arg(arg='b')],
        kwonlyargs=[],
        kw_defaults=[],
        defaults=[]),
      body=[
        Return(
          value=BinOp(
            left=Name(id='a', ctx=Load()),
            op=Add(),
            right=Name(id='b', ctx=Load())))],
      decorator_list=[])],
  type_ignores=[])


## Example 3: View all symbols in a specific file

In [37]:
# Show all definitions in agent_loop.py
file_defs = [d for d in index.definitions if 'agent_loop.py' in d.file_path]
print(index.format_results(file_defs))


|---- agent/agent_loop.py
class AgentConfig:
class Agent:
  def __init__(self, history: HistoryStore, config: AgentConfig | None = None) -> None:
  def _debug_render_md(self, text: str) -> str:
  def _get_debug_theme(self):
  def _debug_prefix(self, role: str | None = None) -> str:
  def _debug_role(self, role: str | None) -> str:
  def _debug_label(self, label: str, *, kind: str = "dim") -> str:
  def reset(self) -> None:
  def dump_context(self) -> str:
  def dump_tools(self, *, as_json: bool = False) -> str:
  def chat(self, user_text: str) -> str:
  def _debug_print_round_header(self, round_idx: int) -> None:
  def _debug_print_request_summary(self) -> None:
  def _debug_print_response_summary(self, assistant_msg: dict[str, Any]) -> None:
  def _truncate(s: str, n: int) -> str:


## Example 4: Inspect a specific definition

In [38]:
# Find the chat method and show details
results = index.search('chat')
if results:
    for definition in results[:3]:  # Show first 3 results
        print(f"Name: {definition.name}")
        print(f"Type: {definition.type}")
        print(f"Signature: {definition.signature}")
        print(f"File: {definition.file_path}")
        print(f"Line: {definition.line_number}")
        if definition.parent:
            print(f"Parent class: {definition.parent}")
        print("---")

Name: chat
Type: method
Signature: def chat(self, user_text: str) -> str:
File: agent/agent_loop.py
Line: 137
Parent class: Agent
---
Name: chat
Type: method
Signature: def chat(self, messages: list[dict[str, Any]], tools: list[dict[str, Any]]) -> dict[str, Any]:
File: agent/llm_openai_compat.py
Line: 19
Parent class: OpenAICompatClient
---
Name: _responses_to_chat_message
Type: method
Signature: def _responses_to_chat_message(resp: dict[str, Any]) -> dict[str, Any]:
File: agent/llm_openai_compat.py
Line: 200
Parent class: OpenAICompatClient
---


In [41]:
code = """
class Calculator:
    PI = 3.14159

    def add(self, a, b):
        return a + b

    def area_of_circle(self, r):
        return self.PI * r * r
"""


In [55]:
import ast

tree = ast.parse(code)
print(ast.dump(tree, indent=2))


Module(
  body=[
    ClassDef(
      name='Calculator',
      bases=[],
      keywords=[],
      body=[
        Assign(
          targets=[
            Name(id='PI', ctx=Store())],
          value=Constant(value=3.14159)),
        FunctionDef(
          name='add',
          args=arguments(
            posonlyargs=[],
            args=[
              arg(arg='self'),
              arg(arg='a'),
              arg(arg='b')],
            kwonlyargs=[],
            kw_defaults=[],
            defaults=[]),
          body=[
            Return(
              value=BinOp(
                left=Name(id='a', ctx=Load()),
                op=Add(),
                right=Name(id='b', ctx=Load())))],
          decorator_list=[]),
        FunctionDef(
          name='area_of_circle',
          args=arguments(
            posonlyargs=[],
            args=[
              arg(arg='self'),
              arg(arg='r')],
            kwonlyargs=[],
            kw_defaults=[],
            defaults=[]),
      

In [51]:
print(code)


class Calculator:
    PI = 3.14159

    def add(self, a, b):
        return a + b

    def area_of_circle(self, r):
        return self.PI * r * r



In [54]:
def print_ast(node, indent=0):
    prefix = "  " * indent
    print(f"{prefix}{type(node).__name__}: {getattr(node, 'name', '')} {'=' if hasattr(node, 'value') else ''} {getattr(node, 'value', '')}")

    for child in ast.iter_child_nodes(node):
        print_ast(child, indent + 1)

print_ast(tree)


Module:   
  ClassDef: Calculator  
    Assign:  = <ast.Constant object at 0x106028be0>
      Name:   
        Store:   
      Constant:  = 3.14159
    FunctionDef: add  
      arguments:   
        arg:   
        arg:   
        arg:   
      Return:  = <ast.BinOp object at 0x106028520>
        BinOp:   
          Name:   
            Load:   
          Add:   
          Name:   
            Load:   
    FunctionDef: area_of_circle  
      arguments:   
        arg:   
        arg:   
      Return:  = <ast.BinOp object at 0x106028310>
        BinOp:   
          BinOp:   
            Attribute:  = <ast.Name object at 0x10602a620>
              Name:   
                Load:   
              Load:   
            Mult:   
            Name:   
              Load:   
          Mult:   
          Name:   
            Load:   


In [44]:
# 4. Pre-order traversal (node first)

"""
['Module', 'ClassDef', 'Assign', 'Name', 'Constant',
 'FunctionDef', 'arguments', 'arg', 'arg', 'Return',
 'BinOp', 'Name', 'Add', 'Name',
 'FunctionDef', 'arguments', 'arg', 'Return',
 'BinOp', 'Attribute', 'Mult', 'Name']
"""

def preorder(node, result):
    result.append(type(node).__name__)
    for child in ast.iter_child_nodes(node):
        preorder(child, result)

pre = []
preorder(tree, pre)
pre

['Module',
 'ClassDef',
 'Assign',
 'Name',
 'Store',
 'Constant',
 'FunctionDef',
 'arguments',
 'arg',
 'arg',
 'arg',
 'Return',
 'BinOp',
 'Name',
 'Load',
 'Add',
 'Name',
 'Load',
 'FunctionDef',
 'arguments',
 'arg',
 'arg',
 'Return',
 'BinOp',
 'BinOp',
 'Attribute',
 'Name',
 'Load',
 'Load',
 'Mult',
 'Name',
 'Load',
 'Mult',
 'Name',
 'Load']

In [46]:
# LLM-ready output you’d generate
def summarize_preorder(node):
    summaries = []

    for n in ast.walk(node):
        if isinstance(n, ast.ClassDef):
            summaries.append(f"class {n.name}")
        elif isinstance(n, ast.FunctionDef):
            args = [a.arg for a in n.args.args]
            summaries.append(f"def {n.name}({', '.join(args)})")

    return summaries

summarize_preorder(tree)



['class Calculator', 'def add(self, a, b)', 'def area_of_circle(self, r)']

In [47]:
# 5. Post-order traversal (children first)
# Code
def postorder(node, result):
    for child in ast.iter_child_nodes(node):
        postorder(child, result)
    result.append(type(node).__name__)

post = []
postorder(tree, post)
post

['Store',
 'Name',
 'Constant',
 'Assign',
 'arg',
 'arg',
 'arg',
 'arguments',
 'Load',
 'Name',
 'Add',
 'Load',
 'Name',
 'BinOp',
 'Return',
 'FunctionDef',
 'arg',
 'arg',
 'arguments',
 'Load',
 'Name',
 'Load',
 'Attribute',
 'Mult',
 'Load',
 'Name',
 'BinOp',
 'Mult',
 'Load',
 'Name',
 'BinOp',
 'Return',
 'FunctionDef',
 'ClassDef',
 'Module']

In [48]:
# LLM-ready semantic extraction
def summarize_postorder(node):
    summaries = []

    for n in ast.walk(node):
        if isinstance(n, ast.Return):
            summaries.append(
                f"returns expression: {ast.dump(n.value, include_attributes=False)}"
            )

    return summaries

summarize_postorder(tree)

["returns expression: BinOp(left=Name(id='a', ctx=Load()), op=Add(), right=Name(id='b', ctx=Load()))",
 "returns expression: BinOp(left=BinOp(left=Attribute(value=Name(id='self', ctx=Load()), attr='PI', ctx=Load()), op=Mult(), right=Name(id='r', ctx=Load())), op=Mult(), right=Name(id='r', ctx=Load()))"]

In [49]:
# 6. Level-order traversal (breadth-first)
# Code
from collections import deque

def level_order(node):
    queue = deque([node])
    result = []

    while queue:
        current = queue.popleft()
        result.append(type(current).__name__)
        for child in ast.iter_child_nodes(current):
            queue.append(child)

    return result

level_order(tree)

['Module',
 'ClassDef',
 'Assign',
 'FunctionDef',
 'FunctionDef',
 'Name',
 'Constant',
 'arguments',
 'Return',
 'arguments',
 'Return',
 'Store',
 'arg',
 'arg',
 'arg',
 'BinOp',
 'arg',
 'arg',
 'BinOp',
 'Name',
 'Add',
 'Name',
 'BinOp',
 'Mult',
 'Name',
 'Load',
 'Load',
 'Attribute',
 'Mult',
 'Name',
 'Load',
 'Name',
 'Load',
 'Load',
 'Load']

## AST Traversal Code Examples

Below are Python code examples demonstrating different AST traversal strategies, inspired by how tools like ast-grep operate.

In [63]:
# Example: Pre-order Traversal (Parent before Children) - Enhanced
import ast

def preorder(node, visit, depth=0):
    visit(node, depth)
    for child in ast.iter_child_nodes(node):
        preorder(child, visit, depth + 1)

def format_node(node, depth=0):
    """Format a node with indentation and useful info"""
    indent = "  " * depth
    node_type = type(node).__name__
    
    # Add specific information based on node type
    if isinstance(node, ast.Name):
        info = f"{node_type}(id='{node.id}')"
    elif isinstance(node, ast.FunctionDef):
        info = f"{node_type}(name='{node.name}')"
    elif isinstance(node, ast.arg):
        info = f"{node_type}(arg='{node.arg}')"
    elif isinstance(node, ast.Constant):
        info = f"{node_type}(value={node.value!r})"
    elif isinstance(node, ast.BinOp):
        op_name = type(node.op).__name__
        info = f"{node_type}(op={op_name})"
    else:
        info = node_type
    
    print(f"{indent}{info}")

source = """class Calculator:
    PI = 3.14159

    def add(self, a, b):
        return a + b

    def area_of_circle(self, r):
        return self.PI * r * r"""
tree = ast.parse(source)
preorder(tree, format_node)

Module
  ClassDef
    Assign
      Name(id='PI')
        Store
      Constant(value=3.14159)
    FunctionDef(name='add')
      arguments
        arg(arg='self')
        arg(arg='a')
        arg(arg='b')
      Return
        BinOp(op=Add)
          Name(id='a')
            Load
          Add
          Name(id='b')
            Load
    FunctionDef(name='area_of_circle')
      arguments
        arg(arg='self')
        arg(arg='r')
      Return
        BinOp(op=Mult)
          BinOp(op=Mult)
            Attribute
              Name(id='self')
                Load
              Load
            Mult
            Name(id='r')
              Load
          Mult
          Name(id='r')
            Load


In [64]:
# Example: Post-order Traversal (Children before Parent) - Enhanced
def postorder(node, visit, depth=0):
    for child in ast.iter_child_nodes(node):
        postorder(child, visit, depth + 1)
    visit(node, depth)

print("\n--- Post-order Traversal ---")
postorder(tree, format_node)


--- Post-order Traversal ---
        Store
      Name(id='PI')
      Constant(value=3.14159)
    Assign
        arg(arg='self')
        arg(arg='a')
        arg(arg='b')
      arguments
            Load
          Name(id='a')
          Add
            Load
          Name(id='b')
        BinOp(op=Add)
      Return
    FunctionDef(name='add')
        arg(arg='self')
        arg(arg='r')
      arguments
                Load
              Name(id='self')
              Load
            Attribute
            Mult
              Load
            Name(id='r')
          BinOp(op=Mult)
          Mult
            Load
          Name(id='r')
        BinOp(op=Mult)
      Return
    FunctionDef(name='area_of_circle')
  ClassDef
Module


In [60]:
# Example: Level-order Traversal (Breadth-first) - Enhanced
from collections import deque

def levelorder(node, visit):
    queue = deque([(node, 0)])  # (node, depth) tuples
    while queue:
        current, depth = queue.popleft()
        visit(current, depth)
        for child in ast.iter_child_nodes(current):
            queue.append((child, depth + 1))

print("\n--- Level-order Traversal ---")
levelorder(tree, format_node)


--- Level-order Traversal ---
Module
  FunctionDef(name='add')
    arguments
    Return
      arg(arg='a')
      arg(arg='b')
      BinOp(op=Add)
        Name(id='a')
        Add
        Name(id='b')
          Load
          Load


## Convert Traversal Output to LLM-Compatible Code

In [65]:
def traversal_to_llm_code(node, traversal_func=preorder, include_metadata=True):
    """
    Convert AST traversal output to LLM-compatible structured representation.
    
    Args:
        node: The AST node to traverse
        traversal_func: The traversal function to use (preorder, postorder, levelorder)
        include_metadata: Whether to include line numbers and other metadata
    
    Returns:
        A structured dictionary representation suitable for LLM processing
    """
    nodes = []
    
    def collect_node_info(n, depth):
        node_info = {
            "type": type(n).__name__,
            "depth": depth,
        }
        
        # Add node-specific attributes
        if isinstance(n, ast.Name):
            node_info["id"] = n.id
            node_info["ctx"] = type(n.ctx).__name__
        elif isinstance(n, ast.FunctionDef):
            node_info["name"] = n.name
            node_info["args"] = [arg.arg for arg in n.args.args]
        elif isinstance(n, ast.ClassDef):
            node_info["name"] = n.name
            node_info["bases"] = [ast.unparse(base) for base in n.bases]
        elif isinstance(n, ast.arg):
            node_info["arg"] = n.arg
        elif isinstance(n, ast.Constant):
            node_info["value"] = n.value
        elif isinstance(n, ast.BinOp):
            node_info["operator"] = type(n.op).__name__
        elif isinstance(n, ast.UnaryOp):
            node_info["operator"] = type(n.op).__name__
        elif isinstance(n, ast.Compare):
            node_info["operators"] = [type(op).__name__ for op in n.ops]
        elif isinstance(n, ast.Call):
            if isinstance(n.func, ast.Name):
                node_info["function"] = n.func.id
            elif isinstance(n.func, ast.Attribute):
                node_info["function"] = n.func.attr
        elif isinstance(n, ast.Attribute):
            node_info["attr"] = n.attr
        elif isinstance(n, ast.Import):
            node_info["names"] = [alias.name for alias in n.names]
        elif isinstance(n, ast.ImportFrom):
            node_info["module"] = n.module
            node_info["names"] = [alias.name for alias in n.names]
        
        # Add metadata if requested
        if include_metadata and hasattr(n, 'lineno'):
            node_info["lineno"] = n.lineno
            if hasattr(n, 'col_offset'):
                node_info["col_offset"] = n.col_offset
        
        nodes.append(node_info)
    
    # Execute the traversal
    traversal_func(node, collect_node_info)
    
    # Create the final structured output
    result = {
        "traversal_type": traversal_func.__name__,
        "total_nodes": len(nodes),
        "nodes": nodes,
        "source_code": ast.unparse(node) if isinstance(node, ast.Module) else None
    }
    
    return result

In [66]:
# Example usage: Convert traversal to LLM-compatible format
import json

result = traversal_to_llm_code(tree, preorder)
print(json.dumps(result, indent=2))

{
  "traversal_type": "preorder",
  "total_nodes": 35,
  "nodes": [
    {
      "type": "Module",
      "depth": 0
    },
    {
      "type": "ClassDef",
      "depth": 1,
      "name": "Calculator",
      "bases": [],
      "lineno": 1,
      "col_offset": 0
    },
    {
      "type": "Assign",
      "depth": 2,
      "lineno": 2,
      "col_offset": 4
    },
    {
      "type": "Name",
      "depth": 3,
      "id": "PI",
      "ctx": "Store",
      "lineno": 2,
      "col_offset": 4
    },
    {
      "type": "Store",
      "depth": 4
    },
    {
      "type": "Constant",
      "depth": 3,
      "value": 3.14159,
      "lineno": 2,
      "col_offset": 9
    },
    {
      "type": "FunctionDef",
      "depth": 2,
      "name": "add",
      "args": [
        "self",
        "a",
        "b"
      ],
      "lineno": 4,
      "col_offset": 4
    },
    {
      "type": "arguments",
      "depth": 3
    },
    {
      "type": "arg",
      "depth": 4,
      "arg": "self",
      "lineno": 4

In [67]:
# Compare different traversal types
print("\n=== Comparing Traversal Types ===\n")
for traversal in [preorder, postorder, levelorder]:
    result = traversal_to_llm_code(tree, traversal, include_metadata=False)
    print(f"{result['traversal_type']} - Total nodes: {result['total_nodes']}")
    print(f"First 3 nodes: {[n['type'] for n in result['nodes'][:3]]}")
    print()


=== Comparing Traversal Types ===

preorder - Total nodes: 35
First 3 nodes: ['Module', 'ClassDef', 'Assign']

postorder - Total nodes: 35
First 3 nodes: ['Store', 'Name', 'Constant']

levelorder - Total nodes: 35
First 3 nodes: ['Module', 'ClassDef', 'Assign']

